# Inputs to the Model

In [ ]:
no_of_training_records =100 # -1 for all recods in the inputs

# Data load from Nimbix File Manager

Update the **username**, **password** and **traning url** for the current nimbix file manager instance

In [ ]:
!pip install pandas
import pandas as pd
testdf = pd.read_csv('/data/CognitiveBanking/data/prediction.csv')
testdf = testdf.head(no_of_training_records)
testdf

In [ ]:

testdf = testdf.head(no_of_training_records)
testdf = testdf.dropna(subset = ['loan_amnt',
                           'funded_amnt',
                           'funded_amnt_inv', 
                           'int_rate',
                           'annual_inc', 
                           'dti',
                           'open_acc', 
                           'revol_bal',
                           'revol_util', 
                           'total_acc',
                           'total_rec_int', 
                           'tot_cur_bal',
                           'total_rev_hi_lim'
                           ])

testdf = testdf.reset_index(drop=True)
testdf

# Directory to fetch model

In [ ]:
from os.path import expanduser
import glob

modelDir = expanduser("~")+'/model'

# Filter the required columns

In [ ]:


inputXtest = testdf.loc[:, ['loan_amnt',
                           'funded_amnt',
                           'funded_amnt_inv', 
                           'int_rate',
                           'annual_inc', 
                           'dti',
                           'open_acc', 
                           'revol_bal',
                           'revol_util', 
                           'total_acc',
                           'total_rec_int', 
                           'tot_cur_bal',
                           'total_rev_hi_lim'
                           ]].as_matrix()

print('Test Data')
print(inputXtest.shape)
print('\n')

# Restore the tensorflow saved model

In [ ]:
import os
import tensorflow as tf
import glob

sess = tf.Session() 

list_of_files = glob.glob(modelDir+'/*') # * means all if need specific format then *.csv
if(len(list_of_files)>0):
    latest_file = max(list_of_files, key=os.path.getctime)
    saver = tf.train.import_meta_graph(latest_file)
    saver.restore(sess,tf.train.latest_checkpoint(modelDir+'/.'))
    graph = tf.get_default_graph()

# Insert the test data for prediction

In [ ]:
import numpy as np
from pandas import Series
X = graph.get_tensor_by_name('X_Input:0')
Y = graph.get_tensor_by_name('Y_Output:0')
clssification = sess.run(Y, feed_dict={X: inputXtest})
outputdf = (pd.DataFrame(clssification)[0]).astype('int64')
testdf['predicted'] = outputdf
pd.options.display.float_format = '{:,.0f}'.format
match=0
for index, row in testdf.iterrows():
    if row['loan_status'] == row['predicted']:
        match=match+1
accuracy =  match*100/testdf.shape[0]       
print("Prediction Accuracy: ",accuracy)          
testdf